In [ ]:
lang_70=15
kurz_70=5
lang_700=91
kurz_700=11

delta = 0.1
  
df_lof_1.columns

df_lof_1["x_sp"] = savgol_filter(df_lof_1["x"], window_length=lang_70, polyorder=3)
df_lof_1["y_sp"] = savgol_filter(df_lof_1["y"], window_length=lang_70, polyorder=3)

df_lof_1["v_x_sp"] = savgol_filter(df_lof_1["x"], window_length=lang_70, polyorder=3, deriv=1, delta=delta)
df_lof_1["v_y_sp"] = savgol_filter(df_lof_1["y"], window_length=lang_70, polyorder=3, deriv=1,  delta=delta)

df_lof_1["a_x_sp"] = savgol_filter(df_lof_1["x"], window_length=lang_70, polyorder=3, deriv=2 ,  delta=delta)
df_lof_1["a_y_sp"] = savgol_filter(df_lof_1["y"], window_length=lang_70, polyorder=3, deriv=2,  delta=delta)


df_lof_1["x_ib"] = savgol_filter(df_lof_1["x"], window_length=kurz_70, polyorder=3)
df_lof_1["y_ib"] = savgol_filter(df_lof_1["y"], window_length=kurz_70, polyorder=3)

df_lof_1["v_x_ib"] = savgol_filter(df_lof_1["x"], window_length=kurz_70, polyorder=3, deriv=1,  delta=delta)
df_lof_1["v_y_ib"] = savgol_filter(df_lof_1["y"], window_length=kurz_70, polyorder=3, deriv=1,  delta=delta)

df_lof_1["a_x_ib"] = savgol_filter(df_lof_1["x"], window_length=kurz_70, polyorder=3, deriv=2,  delta=delta)
df_lof_1["a_y_ib"] = savgol_filter(df_lof_1["y"], window_length=kurz_70, polyorder=3, deriv=2,  delta=delta)

signal_projection(df_lof_1, signal_columns=("a_x_sp", "a_x_ib"), plot=True, plot_2d=True, plot_3d=True) #diverse Signalwerte betrachten

signal_projection(df_lof_1, signal_columns=("x_sp", "y_sp"), plot=True, plot_2d=True, plot_3d=True) #diverse Signalwerte betrachten
signal_projection(df_lof_1, signal_columns=("x_ib", "y_ib"), plot=True, plot_2d=True, plot_3d=True) #diverse Signalwerte betrachten

df_lof_1 = Phi_tangents(df_lof_1, x_col="x_sp", y_col="y_sp")





df_lof_1 = subtract_column_pairs   (
    df_lof_1,
    cols_a=["a_x_ib", "a_y_ib"],
    cols_b=["a_x_sp", "a_y_sp"]
)




df_lof_1 = rotate_acceleration_by_phi_column(  
    df_lof_1,
    a_x_col="a_x_ib_minus_a_x_sp",
    a_y_col="a_y_ib_minus_a_y_sp",
    phi_col="phi_tangent_deg"
) #Rotation der Beschleunigung


signal_projection(df_lof_1, signal_columns=("a_x_ib_minus_a_x_sp_along", "a_x_ib_minus_a_x_sp_perp"), plot=True, plot_2d=True, plot_3d=True) #diverse Signalwerte betrachten



In [ ]:
def _convert_coordinates(df) -> None:
    """
    Transform latitude and longitude coordinates from a dataframe onto a x/y coordinate system.
    Starting with the first coordinates as the new origin (0/0).

    :param self.df: The pandas dataframe with columns "latitude" and "longitude"
    """

    def _compute_z() -> None:
        """
        Rename elevation to 'z'
        new columns:
        # z -> (original: from device) z-coordinate (in meters)
        """

        df.rename(columns={'elevation': 'z'}, inplace=True)

    # Transform columns to numeric columns -> all non-numeric values will be set to nan
    df['latitude'] = pd.to_numeric(df['latitude'], errors="coerce")
    df['longitude'] = pd.to_numeric(df['longitude'], errors="coerce")
    df.dropna(subset=['longitude', 'latitude'], inplace=True)

    # Determine the correct utm zone and the EPSG code based on the hemisphere
    first_lat = df['latitude'].iloc[0]
    first_lon = df['longitude'].iloc[0]
    zone_number = math.floor((first_lon + 180) / 6) + 1
    hemisphere = 'N' if first_lat >= 0 else 'S'
    wgs84 = pyproj.CRS('EPSG:4326')
    utm = f"+proj=utm +zone={zone_number} +datum=WGS84 +units=m +{'south' if hemisphere == 'S' else 'north'}"
    transformer = pyproj.Transformer.from_crs(wgs84, utm, always_xy=True)

    # Transform all coordinates to UTM and compute local coordinates
    df['x'], df['y'] = zip(*df.apply(lambda row: (
        transformer.transform(row['longitude'], row['latitude'])[0] -
        transformer.transform(df['longitude'].iloc[0], df['latitude'].iloc[0])[0],
        transformer.transform(row['longitude'], row['latitude'])[1] -
        transformer.transform(df['longitude'].iloc[0], df['latitude'].iloc[0])[1]
    ), axis=1))

    # Convert elevation
    _compute_z()